In [1]:
# -------------------------
# 1. Imports
# -------------------------
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

sns.set(style='whitegrid')

In [2]:
# -------------------------
# 2. Set Paths
# -------------------------
project_root = os.path.dirname(os.getcwd())
raw_folder = os.path.join(project_root, "data", "raw")
processed_folder = os.path.join(project_root, "data", "processed")
os.makedirs(processed_folder, exist_ok=True)

file_path = os.path.join(raw_folder, "employee_data.csv")
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Raw dataset not found at: {file_path}")

df = pd.read_csv(file_path)
print("Raw dataset loaded.")

FileNotFoundError: Raw dataset not found at: /Users/benjaminnguyen/Employee_attrition/data/raw/employee_data.csv

In [ ]:
# -------------------------
# 3. Target Variable
# -------------------------
target = "Attrition"
if target not in df.columns:
    raise ValueError(f"Target '{target}' not found in dataset.")

# Encode target: 1 = Yes (Attrition), 0 = No
y = df[target].map({'Yes': 1, 'No': 0})
X = df.drop(columns=[target])

print(f"Target distribution:\n{y.value_counts(normalize=True)}")

In [ ]:

# -------------------------
# 4. Identify Column Types
# -------------------------
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(exclude=np.number).columns.tolist()
print(f"Numeric cols: {numeric_cols}")
print(f"Categorical cols: {categorical_cols}")

In [ ]:
# -------------------------
# 5. Outlier Handling
# -------------------------
for col in numeric_cols:
    Q1 = X[col].quantile(0.25)
    Q3 = X[col].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    X[col] = np.clip(X[col], lower, upper)  # cap outliers

# Visualize distributions by target
for col in numeric_cols:
    plt.figure(figsize=(6,4))
    sns.violinplot(x=y, y=X[col])
    plt.title(f"{col} distribution by Attrition")
    plt.show()

In [ ]:
# -------------------------
# 6. Class Imbalance Check
# -------------------------
print("Class distribution after encoding:")
print(y.value_counts(normalize=True))

In [ ]:
# -------------------------
# 7. Missing Value Indicators
# -------------------------
missing_cols = [col for col in X.columns if X[col].isnull().sum() > 0]
for col in missing_cols:
    X[col + "_missing"] = X[col].isnull().astype(int)
print(f"Missing indicators added for: {missing_cols}")

In [ ]:
# -------------------------
# 8. Impute, Encode, Scale
# -------------------------
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols)
    ],
    remainder="passthrough"  # keep missing indicators
)

X_clean = preprocessor.fit_transform(X)

# Build DataFrame with correct column names
encoded_cat_cols = preprocessor.named_transformers_["cat"]["encoder"].get_feature_names_out(categorical_cols)
clean_feature_names = numeric_cols + list(encoded_cat_cols) + [col for col in X.columns if col.endswith("_missing")]
X_clean_df = pd.DataFrame(X_clean, columns=clean_feature_names)

print(f"Final cleaned shape: {X_clean_df.shape}")
X_clean_df.head()


In [ ]:
# -------------------------
# 9. Train/Test Split
# -------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_clean_df, y, test_size=0.20, random_state=42, stratify=y
)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

In [ ]:
# -------------------------
# 10. Save Processed Data
# -------------------------
X_train.to_csv(os.path.join(processed_folder, "train.csv"), index=False)
X_test.to_csv(os.path.join(processed_folder, "test.csv"), index=False)
y_train.to_csv(os.path.join(processed_folder, "y_train.csv"), index=False)
y_test.to_csv(os.path.join(processed_folder, "y_test.csv"), index=False)

print("Processed train/test datasets saved.")